In [1]:
!pip install tensorflow-gpu
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: 
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
!pip install torch

In [2]:
!pip install textstat

You should consider upgrading via the 'c:\python\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install xgboost

In [3]:
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import re
import datetime
import time
import textstat
from bs4 import BeautifulSoup
from collections import Counter
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from gensim.models.keyedvectors import Word2VecKeyedVectors
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
# import xgboost as xgb

In [4]:
import nltk
# nltk.download('punkt') If needed
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from bs4 import BeautifulSoup ## handles html
import re ## provides regular expressions functionality
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [5]:
from tensorflow import keras

In [6]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Input, concatenate, Dense, Embedding, LSTM, GRU, Bidirectional, BatchNormalization, Dropout
from keras.initializers import Constant
from keras.callbacks import EarlyStopping

In [ ]:
#read data
from google.colab import drive
drive.mount('/content/drive')

In [17]:
# train = pd.read_csv("drive/MyDrive/train.csv", sep=",", encoding="utf-8")
train = pd.read_csv("train.csv")

In [18]:
test = pd.read_csv("test.csv")

In [19]:
train_review = pd.read_csv("reviews.csv", sep=",", encoding="utf-8")

In [ ]:
train_review.info()

In [ ]:
train_review.head()

In [40]:
train.columns

In [41]:
train[train.host_id == train.host_id[2]]

In [42]:
train.head()

In [43]:
# Show target distribution
plt.hist(train.price)
plt.xlabel('Price')
plt.title('Price distribution')
plt.show();

In [44]:

train.isnull().sum()/55284 * 100

In [45]:

train['reviews_per_month'].describe()

In [46]:
train_duplicate = train

In [47]:

train_duplicate[train_duplicate.duplicated()]

In [48]:
train.host_response_rate.value_counts()

In [49]:
train['host_response_time'].isnull().sum()

In [50]:
train['zipcode'].mode()

In [51]:
import warnings

warnings.filterwarnings('ignore')

## Cleaning test dataset

In [52]:


test['host_is_superhost'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
test.host_is_superhost[test.host_is_superhost == "t"] = 1 #replacing "t" with 1
test.host_is_superhost[test.host_is_superhost == "f"] = 0 #replacing "f" with 0
test["host_is_superhost"] = test["host_is_superhost"].astype("int64") #converting host_is_superhost to boolean datatype
test['host_has_profile_pic'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
test.host_has_profile_pic[test.host_has_profile_pic == "t"] = 1 #replacing "t" with 1
test.host_has_profile_pic[test.host_has_profile_pic == "f"] = 0 #replacing "f" with 0
test["host_has_profile_pic"] = test["host_has_profile_pic"].astype("int64") #converting host_is_superhost to boolean datatype

test['host_identity_verified'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
test.host_identity_verified[test.host_identity_verified == "t"] = 1 #replacing "t" with 1
test.host_identity_verified[test.host_identity_verified == "f"] = 0 #replacing "f" with 0
test["host_identity_verified"] = test["host_identity_verified"].astype("int64") #converting host_is_superhost to boolean datatyp
test['host_total_listings_count'].fillna(test['host_total_listings_count'].mean(),inplace = True) #replacing NA with mean of 'host_total_listings_count']
test['zipcode'].fillna('E1',inplace = True) #replacing NA with most frequently occuring zipcode 
test['bathrooms'].fillna(test['bathrooms'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 
test['bedrooms'].fillna(test['bedrooms'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 
test['beds'].fillna(test['beds'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 

test['review_scores_rating'].fillna(test['review_scores_rating'].mean(),inplace = True)
test['review_scores_accuracy'].fillna(test['review_scores_accuracy'].mean(),inplace = True)
test['review_scores_cleanliness'].fillna(test['review_scores_cleanliness'].mean(),inplace = True)
test['review_scores_checkin'].fillna(test['review_scores_checkin'].mean(),inplace = True)
test['review_scores_communication'].fillna(test['review_scores_communication'].mean(),inplace = True)
test['review_scores_location'].fillna(test['review_scores_location'].mean(),inplace = True)
test['review_scores_value'].fillna(test['review_scores_value'].mean(),inplace = True)
test['reviews_per_month'].fillna(test['reviews_per_month'].mean(),inplace = True)

test['summary'].fillna('empty',inplace = True)
test["property_type"] = test["property_type"].astype("category") 
test["room_type"] = test["room_type"].astype("category") 
test["bed_type"] = test["bed_type"].astype("category") 
test['host_response_time'].fillna("missing",inplace = True)
test["host_response_time"] = test["host_response_time"].astype("category")



In [53]:
test['summary'].fillna('empty',inplace = True)
test['name'].fillna('empty',inplace = True)
test['space'].fillna('empty',inplace = True)
test['description'].fillna('empty',inplace = True)
test['neighborhood_overview'].fillna('empty',inplace = True)
test['transit'].fillna('empty',inplace = True)
test['house_rules'].fillna('empty',inplace = True)

In [54]:
test["host_since"] = test["host_since"].astype("datetime64[ns]")
test["host_since"].fillna(pd.to_datetime('2015-05-21'),inplace = True)
test['host_days'] = (datetime.datetime.today() - test['host_since'])//np.timedelta64(1,'D')   
test['host_days'] = test['host_days']/365
x_test_dummies = pd.get_dummies(test, columns = ['host_response_time','room_type','bed_type'], drop_first=True)          
x_test_dummies = x_test_dummies.select_dtypes(exclude=["category","datetime64[ns]","object"])
x_test_dummies.drop(["host_id","latitude","longitude"], inplace=True, axis=1)

## Cleaning Train Dataset

In [55]:
train['host_is_superhost'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
train.host_is_superhost[train.host_is_superhost == "t"] = 1 #replacing "t" with 1
train.host_is_superhost[train.host_is_superhost == "f"] = 0 #replacing "f" with 0
train["host_is_superhost"] = train["host_is_superhost"].astype("int64") #converting host_is_superhost to boolean datatype
train['host_has_profile_pic'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
train.host_has_profile_pic[train.host_has_profile_pic == "t"] = 1 #replacing "t" with 1
train.host_has_profile_pic[train.host_has_profile_pic == "f"] = 0 #replacing "f" with 0
train["host_has_profile_pic"] = train["host_has_profile_pic"].astype("int64") #converting host_is_superhost to boolean datatype

train['host_identity_verified'].fillna(0,inplace = True) #replacing NA with 0, assuming 0 is false value. 
train.host_identity_verified[train.host_identity_verified == "t"] = 1 #replacing "t" with 1
train.host_identity_verified[train.host_identity_verified == "f"] = 0 #replacing "f" with 0
train["host_identity_verified"] = train["host_identity_verified"].astype("int64") #converting host_is_superhost to boolean datatyp


In [56]:
train['bathrooms'].isnull().sum()

In [57]:
train['host_total_listings_count'].fillna(train['host_total_listings_count'].mean(),inplace = True) #replacing NA with mean of 'host_total_listings_count']

In [58]:
train['neighbourhood'].mode()


In [59]:
train['neighbourhood'].fillna('City of Westminster',inplace = True) #replacing NA with most frequently occuring neighbourhood

In [60]:
train['zipcode'].mode()

In [61]:
train['zipcode'].fillna('E1',inplace = True) #replacing NA with most frequently occuring zipcode 

In [62]:
train['bathrooms'].fillna(train['bathrooms'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 

In [63]:
train['bedrooms'].fillna(train['bedrooms'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 

In [64]:
train['beds'].fillna(train['beds'].mean(),inplace = True) #replacing NA with most frequently occuring zipcode 

In [65]:
train['review_scores_rating'].fillna(train['review_scores_rating'].mean(),inplace = True)
train['review_scores_accuracy'].fillna(train['review_scores_accuracy'].mean(),inplace = True)
train['review_scores_cleanliness'].fillna(train['review_scores_cleanliness'].mean(),inplace = True)
train['review_scores_checkin'].fillna(train['review_scores_checkin'].mean(),inplace = True)
train['review_scores_communication'].fillna(train['review_scores_communication'].mean(),inplace = True)
train['review_scores_location'].fillna(train['review_scores_location'].mean(),inplace = True)
train['review_scores_value'].fillna(train['review_scores_value'].mean(),inplace = True)
train['reviews_per_month'].fillna(train['reviews_per_month'].mean(),inplace = True)


In [66]:
train['summary'].fillna('empty',inplace = True)
train["property_type"] = train["property_type"].astype("category") 
train["room_type"] = train["room_type"].astype("category") 
train["bed_type"] = train["bed_type"].astype("category") 
train['host_response_time'].fillna("missing",inplace = True)
train["host_response_time"] = train["host_response_time"].astype("category") 


In [67]:
train['summary'].fillna('empty',inplace = True)
train['name'].fillna('empty',inplace = True)
train['space'].fillna('empty',inplace = True)
train['description'].fillna('empty',inplace = True)
train['neighborhood_overview'].fillna('empty',inplace = True)
train['transit'].fillna('empty',inplace = True)
train['house_rules'].fillna('empty',inplace = True)

In [68]:
train["host_since"].mode()

In [69]:
import pickle

In [70]:

train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [71]:
train = pd.read_pickle('./train.pkl')
test = pd.read_pickle('./test.pkl')

## Cleaning Text Data

In [72]:
def remove_whitespace(text):
    """ Function to remove whitespace (tabs, newlines). """
    return ' '.join(text.split())

In [73]:
from bs4 import BeautifulSoup

In [74]:
def remove_punctuation_and_casing(text):
    """
    Function to remove the punctuation, upper casing and words that include
    non-alphanumeric characters.
    """
    chars = '!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    text = text.translate(str.maketrans(chars, ' ' * len(chars)))
    return ' '.join([word.lower() for word in text.split() if word.isalpha()])

In [75]:
from nltk.corpus import stopwords

english_stopwords = stopwords.words('english')

In [76]:
def remove_stopwords(text):
    """ Function to remove stopwords. """
    return ' '.join([word for word in str(text).split() if word not in english_stopwords])

In [77]:
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Helper function that calls the POS tagger for an input word and return a code that can be used for lemmatization"""
    # Extract the first letter of the POS tag (see the above example to understand the output coming from pos_tag)
    tag = nltk.pos_tag([word])[0][1][0].upper()  
    # Dictionary to map these letters to wordnet codes that the lemmatizer understands
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [78]:
def text_cleaning(documents):
    """
    Function for standard NLP pre-processing including removal of html tags,
    whitespaces, non-alphanumeric characters, and stopwords. Emoticons are
    converted to text that reflects their meaning. Words are subject to
    lemmatization using their POS tags.
    """
    cleaned_text = []  # our output will be a list of documents
    lemmatizer = WordNetLemmatizer()
    
    print('Processing input array with {} elements...'.format(documents.shape[0]))
    counter = 0
    
    for doc in documents:
        text = BeautifulSoup(doc).get_text() # remove html content
        text = remove_whitespace(text) # remove whitespaces
        text = remove_punctuation_and_casing(text) # remove punctuation and casing
        text = remove_stopwords(text) # remove stopwords
        text = ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in text.split()]) # lemmatize each word
        
        cleaned_text.append(text)

        if (counter > 0 and counter % 50 == 0):
            print('Processed {} documents'.format(counter))
            
        counter += 1
        
    return cleaned_text


In [79]:
nltk.download('stopwords') ## to identify stopwords 
nltk.download('averaged_perceptron_tagger') ## for part-of-speech tagging (used for lemmatization)
nltk.download('omw-1.4')
nltk.download('wordnet')

# Lemmatize with POS Tag (Parts of Speech tagging)
def get_wordnet_pos(word):
    """Map POS tag to first character for lemmatization"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

## function to clean text data
def clean_reviews(df):
    """ Standard NLP pre-processing chain including removal of html tags, non-alphanumeric characters, and stopwords.
        Words are subject to lemmatization using their POS tags, which are determind using WordNet. 
    """
    reviews = []

    lemmatizer = WordNetLemmatizer()
    
    print('*' * 40)
    print('Cleaning {} movie reviews.'.format(df.shape[0]))
    counter = 0
    for review in df:
        
        # remove html content
        review_text = BeautifulSoup(review).get_text()
        
        # remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
        # tokenize the sentences with all capital letters transformed to lower case
        words = word_tokenize(review_text.lower())
  
        # filter stopwords
        words = [w for w in words if w not in stopwords.words("english")]
        
        # lemmatize each word to its lemma
        lemma_words =[lemmatizer.lemmatize(i, get_wordnet_pos(i)) for i in words]
    
        reviews.append(lemma_words)
              
        if (counter > 0 and counter % 500 == 0):
            print('Processed {} reviews'.format(counter))
            
        counter += 1
        
    print('DONE')
    print('*' * 40)

    return(reviews) 

In [80]:
summary_clean = clean_reviews(train.summary)

In [81]:
train["summary_clean"] = summary_clean

In [82]:
train.summary_clean

In [83]:
train.summary

In [84]:
#summary	space	description, neighborhood_overview	transit	house_rules
#'name','summary','space','description','experiences_offered','neighborhood_overview','transit','house_rules'
text_processed = text_cleaning(train.experiences_offered)
print(text_processed)

# Modeling

### using only numerical data and FNN

In [85]:
train["host_since"].mode()

In [86]:
train["host_since"] = train["host_since"].astype("datetime64[ns]")
train["host_since"].fillna(pd.to_datetime('2015-05-21'),inplace = True)
train['host_days'] = (datetime.datetime.today() - train['host_since'])//np.timedelta64(1,'D')   
train['host_days'] = train['host_days']/365

In [87]:
x_train_dummies = pd.get_dummies(train, columns = ['host_response_time','room_type','bed_type'], drop_first=True)          
x_train_dummies = x_train_dummies.select_dtypes(exclude=["category","datetime64[ns]","object"])
x_train_dummies.drop(["host_id","latitude","longitude"], inplace=True, axis=1)

In [88]:
x_train_dummies["summary_clean"] = summary_clean

In [89]:
x_train_dummies.info()

In [90]:
y = x_train_dummies[['price']]
x_train_dummies.pop("price")

In [91]:
x_train_dummies.drop(["summary_clean"], inplace=True, axis=1)

In [92]:
seed=888
X_train, X_test, y_train, y_test = train_test_split(x_train_dummies, y, test_size=0.30, random_state=seed)
    
print (X_train.shape)
print(y_train.shape) 

In [93]:
nb_units = 128

In [94]:
 y.shape

In [95]:
from tensorflow import keras
from tensorflow.keras import layers

In [96]:
x_train_dummies.shape[1]

In [97]:
inputs = layers.Input(shape=(x_train_dummies.shape[1],))
layer1 = layers.Dense(nb_units,  activation='relu', #activation function
                kernel_initializer=keras.initializers.he_normal(seed=seed), #initialisation of weights, 
                bias_initializer='zeros')(inputs)
layer2 = layers.Dense(nb_units,activation='relu',kernel_initializer=keras.initializers.he_normal(seed=seed),bias_initializer='zeros')(layer1)
predictions = layers.Dense(1, activation='relu')(layer2)

model = keras.Model(inputs=inputs, outputs=predictions)
model.summary()

In [98]:
model.compile(optimizer='rmsprop', 
    loss='mse', 
    metrics=['mae']) 
    
history = model.fit(
    X_train, 
    y_train, 
    epochs = 5, 
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose = 1) 

## make predictions
pred_ff_train = model.predict(X_train)
pred_ff_test = model.predict(X_test)

In [99]:
?model.compile

In [100]:
pred_ff_test = model.predict(X_test)

In [101]:
pred_ff_test = model.predict(x_test_dummies)

In [102]:
test.info()

In [103]:
pred_ff_test.shape

In [104]:
loss = history.history["mae"] 
val_loss = history.history["val_mae"] 
epochs = range(1, len(loss) + 1) 
plt.figure()
plt.plot(epochs, loss, "bo", label="Training MAE")
plt.plot(epochs, val_loss, "b", label="Validation MAE")
plt.title("Training and validation MAE")

plt.legend()
plt.show()

In [105]:
score = model.evaluate(X_test, y_test, verbose=0)

In [106]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [107]:
from IPython.display import FileLink, FileLinks

In [108]:
pred_kaggle = pd.DataFrame(pred_ff_test, columns=['price'], index = test.listing_id)
pred_kaggle.to_csv('C:/Users/ASUS/Desktop/pred_kaggle.csv')

In [109]:
mean_squared_error(y_train,pred_ff_train)
pred_ff_test = model.predict(y_test)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,pred_ff_test))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test,pred_ff_test))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,pred_ff_test)))

## modeling using only summary column and fnn

In [110]:
import io
import re
import string
import pandas as pd
import gensim
from gensim.models import Word2Vec  
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [111]:
Xclean_train, Xclean_val, y_train, y_val = train_test_split(train['summary'], train['price'], test_size = 0.2, random_state = 888)

In [112]:
X_train = Xclean_train.to_numpy()
X_val = Xclean_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

In [113]:
Xclean_train.shape

In [114]:
def our_standardization(text_data):
  lowercase = tf.strings.lower(text_data) # convert to lowercase
  remove_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # remove HTML tags
  pattern_remove_punctuation = '[%s]' % re.escape(string.punctuation) # pattern to remove punctuation
  remove_punct = tf.strings.regex_replace(remove_html, pattern_remove_punctuation, '') # apply pattern
  remove_double_spaces = tf.strings.regex_replace(remove_punct, '\s+', ' ') # remove double space
  return remove_double_spaces

In [115]:
# Define the size of the vocabulary and the max number of words in a sequence
vocab_size = 10000
seq_length = 100

# Create a vectorization layer
vectorize_layer = TextVectorization(
    standardize = our_standardization,
    max_tokens = vocab_size,
    output_sequence_length = seq_length
    )

In [116]:
vectorize_layer.adapt(X_train)

In [ ]:
# Create a simple model to use word embeddings
embedding_dim = 16
model = Sequential([
  vectorize_layer,
  layers.Embedding(vocab_size, embedding_dim, name="embedding"), 
  layers.GlobalAveragePooling1D(), # each sample is reduced to the average of the word embeddings
  layers.Dense(1, activation='relu')
])

In [ ]:
model.compile(optimizer='rmsprop', 
    loss='mse', 
    metrics=['mae']) 

In [ ]:
model.fit(
    X_train,
    y_train,
    validation_data = (X_val, y_val),
    epochs=30,
    verbose=1)

In [ ]:
pred_ff_train = model.predict(X_train)
pred_ff_test = model.predict(X_val)

In [ ]:
pred_ff_test

In [ ]:
pred_ff_test

In [ ]:
y_train

In [ ]:
y_val

In [ ]:
X_val

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_val,pred_ff_test))  
print('Mean Squared Error:', metrics.mean_squared_error(y_val,pred_ff_test))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val,pred_ff_test)))

In [ ]:
test_summary = test['summary'].to_numpy()

In [ ]:
test_summary.shape

In [ ]:
pred_ff_kaggle = model.predict(test_summary)

In [ ]:
pred_kaggle_summary = pd.DataFrame(pred_ff_kaggle, columns=['price'], index = test.listing_id)
pred_kaggle_summary.to_csv('C:/Users/ASUS/Desktop/pred_kaggle_summary.csv')

In [119]:
np.save('X_train',X_train)
np.save('y_train',y_train)
np.save('X_test',X_test)
np.save('y_test',y_test)

In [7]:
X_train = np.load('./X_train.npy',allow_pickle=True)
y_train = np.load('./y_train.npy')
X_test = np.load('./X_test.npy')
y_test = np.load('./y_test.npy')

In [8]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(44227,) (16586, 29) (44227,) (16586, 1)


## LSTM

In [9]:
def our_standardization(text_data):
  lowercase = tf.strings.lower(text_data) # convert to lowercase
  remove_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # remove HTML tags
  pattern_remove_punctuation = '[%s]' % re.escape(string.punctuation) # pattern to remove punctuation
  remove_punct = tf.strings.regex_replace(remove_html, pattern_remove_punctuation, '') # apply pattern
  remove_double_spaces = tf.strings.regex_replace(remove_punct, '\s+', ' ') # remove double space
  return remove_double_spaces

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string


vocab_size = 10000
seq_length = 500

# Create a vectorization layer
vectorize_layer = TextVectorization(
    standardize = our_standardization,
    max_tokens = vocab_size,
    output_sequence_length = seq_length
    )
vectorize_layer.adapt(X_train)
## Transform sequences of words to seq of integers and labels to tensor

NameError: in user code:

    c:\Python\lib\site-packages\keras\engine\base_preprocessing_layer.py:122 adapt_step  *
        self.update_state(data)
    c:\Python\lib\site-packages\keras\layers\preprocessing\text_vectorization.py:338 update_state  **
        self._index_lookup_layer.update_state(self._preprocess(data))
    c:\Python\lib\site-packages\keras\layers\preprocessing\text_vectorization.py:438 _preprocess
        inputs = self._standardize(inputs)
    C:\Users\rohan\AppData\Local\Temp\ipykernel_32196\1857479628.py:2 our_standardization
        lowercase = tf.strings.lower(text_data) # convert to lowercase

    NameError: name 'tf' is not defined


In [ ]:
X_train = vectorize_layer(X_train)
# X_val = vectorize_layer(X_val)

In [ ]:
test_summary = vectorize_layer(test_summary)

In [11]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

RuntimeError: Physical devices cannot be modified after being initialized

In [33]:
# Create model with LSTM

seq_length = 1
emb_size = 32
rnn_units = 16
input = tf.keras.Input(shape=(seq_length,), dtype="int64") 
emb = layers.Embedding(input_dim=vocab_size, output_dim=emb_size)(input) 
x = layers.LSTM(rnn_units,return_sequences=False)(emb)
output = layers.Dense(1, activation="relu")(x)
model = tf.keras.Model(input, output) 

model.compile(optimizer='rmsprop', 
    loss='mse', 
    metrics=['mae'])
model.summary()


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 1, 32)             320000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 323,153
Trainable params: 323,153
Non-trainable params: 0
_________________________________________________________________


In [39]:
y_train = y_train[:len(X_train)]

In [40]:
y_train = tf.convert_to_tensor(y_train)
# y_val = tf.convert_to_tensor(y_val)

In [14]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [23]:
!nvcc --version

In [41]:
story = model.fit(
    X_train, 
    y_train, 
    epochs = 1,
    batch_size = 64,
    verbose=1)

UnimplementedError:  Cast string to int64 is not supported
	 [[node model_5/Cast (defined at C:\Users\rohan\AppData\Local\Temp\ipykernel_32196\3797479271.py:1) ]] [Op:__inference_train_function_13312]

Function call stack:
train_function


In [ ]:
X_train

In [ ]:
pred_ff_train = model.predict(X_train)
pred_ff_test = model.predict(X_val)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_val,pred_ff_test))  
print('Mean Squared Error:', metrics.mean_squared_error(y_val,pred_ff_test))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val,pred_ff_test)))

In [ ]:
pred_ff_train

In [ ]:
pred_ff_test

In [ ]:
pred_ff_kaggle = model.predict(test_summary)

In [ ]:
pred_kaggle_summary = pd.DataFrame(pred_ff_kaggle, columns=['price'], index = test.listing_id)
pred_kaggle_summary.to_csv('C:/Users/ASUS/Desktop/pred_kaggle_summary.csv')

## LSTM

In [20]:
train_desc = train['description'].fillna('empty')
train_ngbr = train['neighborhood_overview'].fillna('empty')

y_train = train['price']


print(len(train_desc), len(train_ngbr), len(y_train))


def our_standardization(text_data):
  lowercase = tf.strings.lower(text_data) # convert to lowercase
  remove_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # remove HTML tags
  pattern_remove_punctuation = '[%s]' % re.escape(string.punctuation) # pattern to remove punctuation
  remove_punct = tf.strings.regex_replace(remove_html, pattern_remove_punctuation, '') # apply pattern
  remove_double_spaces = tf.strings.regex_replace(remove_punct, '\s+', ' ') # remove double space
  return remove_double_spaces


from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vocab_size = 10000
seq_length = 500

# Create a vectorization layer
vectorize_layer = TextVectorization(
    standardize = our_standardization,
    max_tokens = vocab_size,
    output_sequence_length = seq_length
    )

emb_size = 32
rnn_units = 16


def create_text_model(text_list):
    model = Sequential()
    model.add(Input(shape=(1,), dtype=tf.string))
    vectorize_layer.adapt(text_list)
    model.add(vectorize_layer)
    model.add(Embedding(input_dim=vocab_size, output_dim=emb_size))
    model.add(LSTM(rnn_units, return_sequences=False))
    model.add(Dense(1, activation="relu"))
    return model

def create_combined_model(X):
    X = Flatten()(X)
    X = Dense(1, activation="linear")(X)
    print(X.shape)
    return X

def create_model():
    
    ### Initialize Input layers
    input_desc = Input(shape=(1,), dtype=tf.string)
    input_ngbr = Input(shape=(1,), dtype=tf.string)
    
    ### Create Vectorisation models from text features
    desc_model = create_text_model(train_desc)
    ngbr_model = create_text_model(train_ngbr)
#     combined_model = create_combined_model()
    
    ### Create Data flow
    emb_desc = desc_model(input_desc)
    emb_ngbr = ngbr_model(input_ngbr)
    concat_combined = Concatenate()([emb_desc,emb_ngbr])
    print(concat_combined.shape)
    output = create_combined_model(concat_combined)
    
    ### Finalize the model
    model = Model(inputs = [input_desc, input_ngbr], outputs = output)
    model.compile(optimizer='adam',loss='mse', metrics=['mae'])
    return model

model = create_model()


print(model.summary())

story = model.fit(
    [train_desc, train_ngbr],
    y_train,
    validation_split=0.2,
    epochs = 5,
    batch_size = 32,
    verbose =1)

In [21]:
def our_standardization(text_data):
  lowercase = tf.strings.lower(text_data) # convert to lowercase
  remove_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # remove HTML tags
  pattern_remove_punctuation = '[%s]' % re.escape(string.punctuation) # pattern to remove punctuation
  remove_punct = tf.strings.regex_replace(remove_html, pattern_remove_punctuation, '') # apply pattern
  remove_double_spaces = tf.strings.regex_replace(remove_punct, '\s+', ' ') # remove double space
  return remove_double_spaces

In [42]:
vocab_size = 10000
seq_length = 500

# Create a vectorization layer
vectorize_layer = TextVectorization(
    standardize = our_standardization,
    max_tokens = vocab_size,
    output_sequence_length = seq_length
    )

In [47]:
try:
    vectorize_layer.adapt(train_desc)
    vectorize_layer.adapt(train_ngbr)
    # train_desc = vectorize_layer(train_desc)
    # train_ngbr = vectorize_layer(train_ngbr)
except:
    pass

In [25]:
from tensorflow.keras.layers import Bidirectional

In [ ]:
!nvidia-smi

In [44]:
# Create model with LSTM
emb_size = 100
rnn_units = 64

input_ngbr = tf.keras.Input(shape=(seq_length,), dtype="int64") 
input_desc = tf.keras.Input(shape=(seq_length,), dtype="int64") 
emb_desc = layers.Embedding(input_dim=vocab_size, output_dim=emb_size)(input_desc) 
x_desc = layers.GRU(rnn_units)(emb_desc)
dense_desc = layers.Dense(1, activation="relu")(x_desc)

emb_ngbr = layers.Embedding(input_dim=vocab_size, output_dim=emb_size)(input_ngbr) 
x_ngbr = layers.GRU(rnn_units)(emb_ngbr)
dense_ngbr = layers.Dense(1, activation="relu")(x_ngbr)

concat = concatenate([dense_desc, dense_ngbr])
norm = layers.BatchNormalization()(concat)
dense_full = Dense(128, activation="relu")(norm)
#dense_full = Dense(64, activation="relu")(dense_full)
output_layer = Dense(1, activation="relu")(dense_full)


model = tf.keras.Model(inputs = [input_desc, input_ngbr], outputs = output_layer) 

model.compile(optimizer='adam', 
    loss='mse', 
    metrics=['mae']) 
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 500, 100)     1000000     input_10[0][0]                   
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 500, 100)     1000000     input_9[0][0]                    
____________________________________________________________________________________________

In [46]:
story = model.fit(
   x = [train_desc,train_ngbr], 
   y = y_train, 
    epochs = 1,
    batch_size = 64,verbose =1)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'float\'>", "<class \'str\'>"})'}), <class 'tensorflow.python.framework.ops.EagerTensor'>

In [ ]:
?model.fit

In [ ]:
preds = pd.DataFrame(model.predict([train_desc,train_ngbr]), columns=["price"], index=train.index)

In [ ]:
preds

In [ ]:
?model.predict

In [ ]:
pred_ff_train = model.predict(X_train)
pred_ff_test = model.predict(X_val)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_val,pred_ff_test))  
print('Mean Squared Error:', metrics.mean_squared_error(y_val,pred_ff_test))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val,pred_ff_test)))

In [ ]:
test_summary = vectorize_layer(test_summary)


In [ ]:
pred_ff_kaggle = model.predict(test_summary)

In [ ]:
pred_ff_kaggle

In [ ]:
pred_ff_train

In [ ]:
vocab_size = 10000
seq_length = 500
input = tf.keras.Input(shape=(seq_length,), dtype="int64") 
emb = layers.Embedding(input_dim=vocab_size, output_dim=emb_size, mask_zero=True)(input) 
x = layers.Bidirectional(layers.LSTM(rnn_units))(emb)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="relu")(x)
model = tf.keras.Model(input, output) 

model.compile(optimizer='rmsprop', 
    loss='mse', 
    metrics=['mae']) 
model.summary()

In [ ]:
model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),
    epochs = 2,
    batch_size = 128)